In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
#import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix

GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number
show_animation = True

In [2]:
!rm -f Data_Processor.pyc

In [3]:
import numpy as np
import math
T=5
NX=4
NU=2
DT=0.2
WB=5
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None
    def array(self):
        return np.array([self.x,self.y,self.v,self.yaw],dtype=np.float64)
class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=State(x=state.x, y=state.y, yaw=state.yaw, v=state.v)
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        #print(curr_state.v,a,DT,next_v)
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)
    
    def _get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER
            
        A, B, C = get_linear_model_matrix(curr_state.v, curr_state.yaw, delta)
        
        X=np.array([curr_state.x,curr_state.y,curr_state.v,curr_state.yaw],dtype=np.float64)
        U=np.array([a,delta],dtype=np.float64)
        [next_x,next_y,next_v,next_yaw]=np.matmul(A,X) + np.matmul(B,U)+ C
        
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw),[A,B,C]


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion._get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


    
def get_linear_model_matrix(v, phi, delta):
        A = np.zeros((NX, NX))
        A[0, 0] = 1.0
        A[1, 1] = 1.0
        A[2, 2] = 1.0
        A[3, 3] = 1.0
        A[0, 2] = DT * math.cos(phi)
        A[0, 3] = - DT * v * math.sin(phi)#
        A[1, 2] = DT * math.sin(phi)
        A[1, 3] = DT * v * math.cos(phi)#
        A[3, 2] = DT * math.tan(delta) / WB

        B = np.zeros((NX, NU))
        B[2, 0] = DT
        B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

        C = np.zeros(NX)
        C[0] = DT * v * math.sin(phi) * phi
        C[1] = - DT * v * math.cos(phi) * phi
        C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
        #C = np.zeros(NX)
        return A, B, C
    
def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
def calc_cost( x, u,xref,t,_B):
        
        R = np.diag([0.01, 0.01])  # input cost matrix
        Rd = np.diag([0.01, 1.0])  # input difference cost matrix
        Q = np.diag([1.0, 1.0, 0.5, 0.1])  # state cost matrix
        Qf = Q  # state final matrix
        

        #cost += cvxpy.quad_form(u[:, t], R)
        #print(u,R)
        l = np_quad_form(u, R) + np_quad_form(xref - x, Q)
        
        
        #if t < (T - 1):
        #    l = np_quad_form(u[:, t + 1] - u[:, t], Rd)
            
        
        #l = np_quad_form(xref[:, T] - x[:, T], Qf)


        # compute derivatives of cost
        
        l_x1=2*Q[0,0]*(xref[0]-x[0])*(-1)
        l_x2=2*Q[1,1]*(xref[1]-x[1])*(-1)
        l_x3=2*Q[2,2]*(xref[2]-x[2])*(-1)
        l_x4=2*Q[3,3]*(xref[3]-x[3])*(-1)
            
        l_x=np.array([l_x1,l_x2,l_x3,l_x4])
        l_xx=2 * Q
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(xref-x)
        #x=Ax+Bu+C
        #dx/du=d(B*u)=B
        l_u1 = 2. * u[0] * R[0,0] + np.sum(2*Qdiag*error*(-1)*_B[:,0])
        l_u2 = 2. * u[1] * R[1,1] + np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        l_u = np.array([l_u1, l_u2])

        l_uu = 2. * R
        
        l_ux = np.zeros((NU, NX))

        return l, l_x, l_xx, l_u, l_uu, l_ux

In [4]:
def sim_n_steps(x0,U,xref):
    X=np.zeros((NX,T+1),dtype=np.float64)
    X[:,0]=x0
    _MS=Motion()
    total_grad=0
    for t in range(T):
        _state=State(x=X[0,t],y=X[1,t],v=X[2,t],yaw=X[3,t])
        _MS.set_state(_state)
        nst,Const=_MS._get_next_state(U[0,t],U[1,t])
        B=Const[1]
        x_t1=[nst.x,nst.y,nst.v,nst.yaw]
        u_t=U[:,t]
        xref_t=xref[:,t]
        cc=calc_cost(x_t1,u_t,xref_t,t,B)
        du=0.2*cc[3]
        if t <T-1 : U[:,t+1]-=du
        X[:,t+1]=x_t1.copy()
        #print(X,cc[0],cc[3])
        total_grad+=du
    #return total_grad,U.copy()
    return X,cc,U

In [5]:
import numpy as np
from copy import copy, deepcopy



class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=50
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
    def ilqr(self, x0, U,xref): 
        #U = np.zeros((NX,T),dtype=np.float64) if U is None else U
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                # simulate forward using the current control trajectory
                X, cost, _ = sim_n_steps(x0, U,xref)
                oldcost = copy(cost) # copy for exit condition check

                # 
                f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
                f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
                # for storing quadratized cost function 

                l = np.zeros((self.tN,1)) # immediate state cost 
                l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
                l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
                l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
                l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
                l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
                # for everything except final state
                for t in range(self.tN):
                    # x(t+1) = f(x(t), u(t)) = x(t) + dx(t) * dt
                    # linearized dx(t) = np.dot(A(t), x(t)) + np.dot(B(t), u(t))
                    # f_x = (np.eye + A(t)) * dt
                    # f_u = (B(t)) * dt
                    # continuous --> discrete
                    A, B, C = get_linear_model_matrix(X[2,t],X[3,t], U[1,t])
                    f_x[t] = A
                    f_u[t] = B 

                    
                    (l[t], l_x[t], l_xx[t], l_u[t], l_uu[t], l_ux[t]) = calc_cost(X[:,t], U[:,t],xref[:,t],t,B)
                    

                # and for final state
                #l[-1], l_x[-1], l_xx[-1] = self.cost_final(X[-1])

                sim_new_trajectory = False

            V = l[-1].copy() # value function
            V_x = l_x[-1].copy() # dV / dx
            V_xx = l_xx[-1].copy() # d^2 V / dx^2
            k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
            K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

            # work backwards to solve for V, Q, k, and K
            for t in range(self.tN-2, -1, -1):
                
                Q_x = l_x[t] + np.dot(f_x[t].T, V_x) 
                Q_u = l_u[t] + np.dot(f_u[t].T, V_x)

                Q_xx = l_xx[t] + np.dot(f_x[t].T, np.dot(V_xx, f_x[t])) 
                Q_ux = l_ux[t] + np.dot(f_u[t].T, np.dot(V_xx, f_x[t]))
                Q_uu = l_uu[t] + np.dot(f_u[t].T, np.dot(V_xx, f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))

                k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(K[t].T, np.dot(Q_uu, k[t]))
                V_xx = Q_xx - np.dot(K[t].T, np.dot(Q_uu, K[t]))

            U_new = np.zeros((self.INPUT_SIZE,self.tN))
            x_new = x0.copy()
            for t in range(self.tN - 1): 
                # use feedforward (k) and feedback (K) gain matrices 
                # calculated from our value function approximation
                U_new[:,t] = U[:,t] + k[t] + np.dot(K[t], x_new - X[:,t])
                #_,x_new = self.plant_dynamics(x_new, U_new[t])

            # evaluate the new trajectory 
            X_new, cost_new , _= sim_n_steps(x0, U_new,xref)

            if cost_new[0] < cost[0]: 
                # decrease lambda (get closer to Newton's method)
                lamb /= self.lamb_factor

                X = copy(X_new) # update trajectory 
                U = copy(U_new) # update control signal
                oldcost = copy(cost)
                cost = copy(cost_new)

                sim_new_trajectory = True # do another rollout

                # check to see if update is small enough to exit
                print('...',cost[0],oldcost[0])
                if ii > 0 and ((abs(oldcost[0]-cost[0])/cost[0]) < self.eps_converge):
                    #print("Converged at iteration = %d; Cost = %.4f;"%(ii,cost_new) + 
                    #        " logLambda = %.1f"%np.log(lamb))
                    print("break 1",ii)
                    break

            else: 
                # increase lambda (get closer to gradient descent)
                lamb *= self.lamb_factor
                # print("cost: %.4f, increasing lambda to %.4f")%(cost, lamb)
                if lamb > self.lamb_max: 
                    #print("lambda > max_lambda at iteration = %d;"%ii + 
                    #    " Cost = %.4f; logLambda = %.1f"%(cost, np.log(lamb)))
                    print("break 2",ii)
                    break

        return X, U, cost
    

In [6]:
from Car_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=False
max_len=350
qlen=25
car_move=True
Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen)
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
img=init_state[3]
Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
#print(init_state)

Track generation: 1190..1491 -> 301-tiles track


In [8]:
if rec==True:
    import os
    if not os.path.exists('Rec/'):
        os.mkdir('Rec')
    if not os.path.exists('Fig/'):
        os.mkdir('Fig')


In [ ]:
import matplotlib.gridspec as gridspec
phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        _full_state = np.array([state.x, state.y, state.v, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=X[2,1]
        oyaw=X[3,1]
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(_index)+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(45)
        max_accl=5
        if odelta is not None:
            if abs(U[1,1])<max_steer:
                  di= U[1,1]
            else:
                di=(U[1,1]/abs(U[1,1]))*max_steer
            
            if abs(U[0,1])<max_accl:
                  ai=U[0,1]
            else:
                ai=U[0,1]/abs(U[0,1])*max_accl
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        Motion_Status.set_state(next_state)
        
        log_state.append(_full_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(next_state)
        
        time = time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.4,'xref : '+xref_str,fontsize=10,color='b')
            #axes[0].text(.01,.2,'xbar : '+xbar_str,fontsize=10,color='c')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if itr%25==0:
                plt.savefig('Fig/'+str(int(itr/25))+'.jpg')
            itr+=1
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


... 151.59847335920412 205.246035815519
... 122.91327295030891 151.52645617539298
... 107.2007330699753 123.0757885915847
... 98.0791339291303 107.33583254225488
... 92.3886238075358 98.1098894245132
... 88.53318747122934 92.31179382118353
... 85.73648593365917 88.3720244475356
... 83.56948274597794 85.51717099189179
... 82.14976035509292 83.31839450925108
... 41.575179883968836 41.96749885768193
... 40.52182556323166 41.79630653664048
... 87.06275743917715 87.13009415201616
... 85.59181770078155 86.04776036722726
... 83.65365064256436 86.81763821920026
... 70.77983720550436 71.42797419758762
... 70.99664453713473 71.64671065362566
... 69.58344430160703 71.85257972058882
... 66.8642227985437 70.46269528609334
... 58.429001445294 59.12880985141452
... 55.665688012694 59.468451697661976
... 115.38826345871944 115.4734117685684
... 113.80486904101629 116.2087768455848
... 113.06767021436421 114.62192579167134
... 112.65340427129507 113.88226662793478
... 104.61893099274174 113.46694424612

... 64.74429813612473 66.0228119590848
... 64.28020318955123 65.51589799244687
... 64.55458059054925 65.11215208546588
... 64.93907029479415 65.03909216212374
... 64.36595097684777 65.69714741697872
... 64.6963478971419 65.17594192525831
... 64.75655551487993 65.47100043355071
... 64.00147562114482 65.54130940734588
... 63.15602483552794 64.53385449387514
... 62.67648687254965 63.85707437665806
... 62.368744597528334 63.20168454762792
... 61.783033931531236 62.81890746784796
... 62.23693790838851 62.49498418772995
... 59.53622947628138 60.47728438038733
... 115.819360226001 118.39070061724277
... 109.7693435301729 116.73173138244121
... 105.47454207622263 110.83736122690168
... 101.9259768254137 106.39011793731979
... 101.31860928447443 102.40968118338552
... 98.4736416508624 103.24803945675802
... 97.96648687481058 99.29092092353181
... 74.3078364932028 76.29520102243463
... 72.94655113468245 73.56391641509997
... 72.25391000025542 74.88609771638406
... 70.08795682979817 72.4552722317

... 63.81587051439632 66.1249946599108
... 63.413739062040676 64.94355803398513
... 62.250895110273504 64.8916253639351
... 62.085800214398276 63.055838770288844
... 60.990020967259895 63.31914942188861
... 61.69104026928278 62.10954655552943
... 60.78117512542821 62.931670509420016
... 61.5611212725203 61.90623669551558
... 60.712143133380344 62.80385322243899
... 61.51057434868173 61.838858134667426
... 60.69616213725058 62.75221239443954
... 61.48779352402142 61.82334794947817
... 60.70004922228196 62.727033154572034
... 61.47303734167639 61.827526876761915
... 60.71085689471933 62.70934321336605
... 61.459060464770694 61.838799367844175
... 60.723686155106854 62.692220175831665
... 61.44357042400906 61.85236162997237
... 60.73674317123597 62.67351401808054
... 61.426197522277185 61.86641899918101
... 60.749488111256014 62.65292485960792
... 61.407222507073044 61.88039453561286
... 60.761800400921715 62.63077452526552
... 61.38712329774601 61.89412187271974
... 60.77369771935395 62.

... 57.11231910256001 58.74038121179416
... 57.666519845975344 58.10216760683836
... 57.10653385137371 58.82506956389588
... 57.74217247008142 58.085014882280824
... 57.094531075132025 58.91603639499434
... 57.824047259230916 58.060611053572316
... 57.076678967671796 59.016321924782986
... 57.91539303146354 58.02909354725868
... 57.05355888836633 59.130024752548714
... 56.25590535390092 57.218607800673944
... 56.05221402079918 57.29110907142783
... 56.09859424004316 57.05935474385842
... 55.95993341564904 57.137967541006326
... 56.04262429298692 56.970288285634275
... 55.933769658169666 57.082845769167825
... 56.02890436210605 56.94526400371161
... 55.93449168024165 57.068610354504095
... 56.03165518989009 56.946340815685886
... 55.94513949500412 57.070343494814374
... 56.03992118766328 56.9571038493612
... 55.95860792443469 57.077458542343145
... 56.04925180344893 56.970664514005676
... 55.972080403962075 57.08565591095658
... 56.05798930420301 56.9842816791935
... 55.98456085967591 5

... 57.26954517591046 58.297091945141574
... 113.27596277024178 117.90213847523708
... 105.90550978483267 115.35517321165612
... 94.31211480221424 95.6887260639256
... 92.42071686012514 96.74740133285349
... 81.0234280059951 82.0148564838826
... 77.31396370130595 82.69319672919991
... 72.7982794996521 73.44529733586434
... 69.19029970559893 74.78877520146278
... 66.00523039820588 66.50035280718949
... 63.14162034675412 67.29826733840821
... 62.757025367644154 63.17562472341607
... 60.53737953972997 64.40049479985892
... 55.49706883561182 57.2811604638224
... 55.24346255386159 56.41655627402281
... 54.36385413532145 56.51729359126725
... 111.80222167102745 115.68698437960367
... 107.29056199486422 112.68215377225688
... 101.82617868713196 109.98733652389626
... 94.68799918468575 97.50338269040667
... 90.82387348811645 96.96894226078535
... 85.88570660093026 89.53035156669594
... 85.34923262745367 87.53436193563984
... 85.95170920465021 87.29933801455871
... 84.59363501075387 85.79540629

... 117.87087719384613 118.07349961038086
... 110.82737240889688 118.5679932237296
... 101.10089668253477 101.124570318584
... 95.02167223036503 101.90159393729877
... 93.64153947999219 96.06661971133781
... 89.79828872108185 93.84791254938119
... 90.01735728979813 91.03793755345451
... 83.79794077520155 84.8554144348393
... 83.0879649265347 84.7885792132892
... 81.691008607721 84.96436680891256
... 80.47451234746711 82.28888297227434
... 81.6439725971616 81.6881003012204
break 1 3
... 78.17001148300692 79.22432537701708
... 70.98617429575482 71.2298959734099
... 71.0703000571231 71.88349176074573
... 71.09802452821037 71.95568034703429
... 69.8544024133524 71.97315097245767
... 70.63012375855357 71.76776464939566
... 70.43496760089155 71.35801767935368
... 68.31863593027185 71.1579614278538
... 65.67292287101995 69.06970124740296
... 65.87001536785739 66.48519716673054
... 65.23989872215935 66.67234125375205
... 63.450256488749716 65.66322735470601
... 63.84337729183579 64.16035121083

... 86.04359955449414 87.17415177904783
... 79.2958388270312 79.60176405844074
... 79.30901473766164 80.10732172609025
... 79.57818098216066 80.11485344401329
... 77.49936021762642 80.37663706747207
... 75.24854581592487 78.36655786055701
... 73.31370595797576 76.12716839569367
... 73.19903242452611 74.20210721318934
... 73.27728097306212 74.074002934381
... 72.78808205134173 74.03268458511604
... 72.10711547327934 73.28871636879498
... 67.13453854403805 67.88750428753484
... 66.7491910448049 68.44823097737864
... 66.72707505523307 67.29107303695801
... 66.61560719793282 67.49694623616664
... 67.38740560225025 67.4270096644588
break 1 2
... 65.75723210683267 66.12046543218285
... 65.68125759565929 66.53220972711641
... 65.80803529218839 66.38023141990212
... 66.16019633214833 66.99759417602762
... 66.32971226948338 66.88482766480746
... 65.0356577791522 67.00825034565621
... 65.38231208034075 65.78114853224334
... 65.90342639292078 66.12204159622392
... 65.83179653488266 66.64085417936

... 61.62388668633356 62.47719822229939
... 60.76360815268514 62.848735385267524
... 61.69543004544237 61.84621975390235
... 60.47925582324589 62.97462314126057
... 115.74774233783377 119.32723525234088
... 106.67382441852932 118.95571590157684
... 104.016331179127 107.75086870130221
... 97.49060405702542 99.56083806147451
... 93.0800837097779 99.77804416178805
... 86.37611376598855 88.3374804119221
... 83.83372827565103 88.33158473006928
... 80.86565478129272 83.07371436839836
... 79.7150955421532 82.57637595898312
... 80.54177229818285 81.25355283685755
... 78.53743434456293 82.5824632092689
... 77.04090338785569 77.97976154165968
... 75.27945790373705 78.88429218416059
... 72.96362405296095 74.88667972238892
... 72.44325266800047 74.45835248730617
... 72.34630582200235 73.91515376042778
... 71.9218248857642 73.88620909998973
... 72.39300831398363 73.35873417821787
... 71.55784628282092 74.03128712877742
... 70.76228090630327 72.25767109287439
... 70.51471454255606 72.22703697836141


In [ ]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state
}

In [ ]:
f=open('dataset','rb')
dataset=pickle.load(f)
f.close()

In [ ]:
dataset

In [ ]:
np.arctan2(1,1)

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 1
n = 1
np.random.seed(1)
A = 10#np.random.randn(m, n)
b = 5#np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
print(x.value)

In [ ]:
x=np.array(range(0,100))
x=x/100

In [ ]:
y=10*x-5
y=np.square(y)
plt.plot(x,y)
plt.show()

In [ ]:
import gym
gym.make('CarRacing-v0')